In [1]:
import os 
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [2]:
import torch 
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize 


from pathlib import Path 
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader

In [3]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 22.3 MB/s 
     |████████████████████████████████| 5.3 MB 42.7 MB/s 
     |████████████████████████████████| 212 kB 52.9 MB/s 
     |████████████████████████████████| 115 kB 73.2 MB/s 
     |████████████████████████████████| 163 kB 77.4 MB/s 
     |████████████████████████████████| 127 kB 73.7 MB/s 
     |████████████████████████████████| 7.6 MB 56.6 MB/s 
     |████████████████████████████████| 115 kB 77.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [4]:
import datasets
sst = datasets.load_dataset('sst2')

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset sst2 downloaded and prepared to /root/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
from transformers import GPT2Tokenizer, set_seed
set_seed(42)
tokenizer = GPT2Tokenizer.from_pretrained("sshleifer/tiny-gpt2")
#tokenizer.pad_token = tokenizer.eos_token
#tokenizer.padding_side = "left" # Very Important
tokenizer.pad_token = tokenizer.eos_token
# if tokenizer.pad_token is None:
#     tokenizer.add_special_tokens({'pad_token': '[PAD]'})

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)


tokenized_datasets = sst.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["idx","sentence"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/662 [00:00<?, ?B/s]

  0%|          | 0/68 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [6]:
# def label_(examples):
#   return {'labels':(examples['labels']).unsqueeze(0)}
# tokenized_datasets = tokenized_datasets.map(label_)

In [7]:
train_dataset = tokenized_datasets['train'].shuffle(seed=42).select(range(960))
val_dataset = tokenized_datasets['validation'].shuffle(seed=42).select(range(240))

In [8]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


In [9]:
vocab_size = tokenizer.vocab_size

In [10]:
from transformers import GPT2ForSequenceClassification, GPT2Config
model_config = GPT2Config.from_pretrained("gpt2", num_labels=2, vocab_size=vocab_size)
model = GPT2ForSequenceClassification.from_pretrained("gpt2", config=model_config)
#model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = model.config.eos_token_id

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [12]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  after removing the cwd from sys.path.


In [13]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [14]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [15]:
training_args.num_train_epochs = 3.0
training_args.per_device_eval_batch_size=4
training_args.per_device_train_batch_size=4

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

In [17]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 960
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 720


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.475526,0.850000
2,No log,0.613385,0.850000
3,0.585400,0.729979,0.870833


***** Running Evaluation *****
  Num examples = 240
  Batch size = 4
***** Running Evaluation *****
  Num examples = 240
  Batch size = 4
Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 240
  Batch size = 4


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=720, training_loss=0.48465179867214625, metrics={'train_runtime': 842.5957, 'train_samples_per_second': 3.418, 'train_steps_per_second': 0.855, 'total_flos': 1505069283409920.0, 'train_loss': 0.48465179867214625, 'epoch': 3.0})

In [18]:
from transformers import GPT2ForSequenceClassification, GPT2Config
model_config = GPT2Config.from_pretrained("sshleifer/tiny-gpt2", num_labels=2, vocab_size=vocab_size)
model = GPT2ForSequenceClassification.from_pretrained("sshleifer/tiny-gpt2", config=model_config)
#model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = model.config.eos_token_id

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--sshleifer--tiny-gpt2/snapshots/5f91d94bd9cd7190a9f3216ff93cd1dd95f2c7be/config.json
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 2,
  "n_head": 2,
  "n_inner": null,
  "n_layer": 2,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.23.1",

Downloading:   0%|          | 0.00/2.51M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--sshleifer--tiny-gpt2/snapshots/5f91d94bd9cd7190a9f3216ff93cd1dd95f2c7be/pytorch_model.bin
Some weights of the model checkpoint at sshleifer/tiny-gpt2 were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at sshleifer/tiny-gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN

In [19]:
training_args.num_train_epochs = 10.0
training_args.per_device_eval_batch_size=8
training_args.per_device_train_batch_size=8

In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

In [21]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 960
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1200


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.694784,0.462500
2,No log,0.693994,0.491667
3,No log,0.693811,0.491667
4,No log,0.693882,0.487500
5,0.693000,0.693642,0.504167
6,0.693000,0.693392,0.495833
7,0.693000,0.693467,0.508333
8,0.693000,0.693365,0.508333
9,0.689900,0.693338,0.508333
10,0.689900,0.693329,0.508333


***** Running Evaluation *****
  Num examples = 240
  Batch size = 8
***** Running Evaluation *****
  Num examples = 240
  Batch size = 8
***** Running Evaluation *****
  Num examples = 240
  Batch size = 8
***** Running Evaluation *****
  Num examples = 240
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 240
  Batch size = 8
***** Running Evaluation *****
  Num examples = 240
  Batch size = 8
***** Running Evaluation *****
  Num examples = 240
  Batch size = 8
***** Running Evaluation *****
  Num examples = 240
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 240
  Batch size = 8
**

TrainOutput(global_step=1200, training_loss=0.6910861333211263, metrics={'train_runtime': 50.6766, 'train_samples_per_second': 189.436, 'train_steps_per_second': 23.68, 'total_flos': 9201254400.0, 'train_loss': 0.6910861333211263, 'epoch': 10.0})